In [ ]:
from xml.etree import ElementTree as ET
import os
import statistics

In [ ]:
# 1年分の各種スタッツを取得するためのコード

# フォルダパス
folder_path = "xml_files/2023"   # 1年分のxmlファイルが保存されているフォルダを指定

# カウントするスタッツの宣言と初期値
total_rounds = 0     # 局数
total_furo = 0       # 副露数（２副露目以降はカウントしない）
total_reach = 0      # リーチ数（宣言数）
total_agari = 0      # アガリ数（ダブロンは2件）
total_houju = 0      # 放銃数（ダブロンは1件）
total_ryuukyoku = 0  # 流局数（すべての流局を含む）
datens = []          # アガリ打点（詰み棒・供託を除く素点）
player_dans = []     # 対局者の段位情報（16:7段, 17:8段, 18:9段, 19:10段, 20:天鳳位）
player_rates = []    # 対局者のレート情報

# 対局数（xmlファイルの数をカウント）
total_games = len([filename for filename in os.listdir(folder_path) if filename.endswith(".xml")])

# フォルダ内のすべてのXMLファイルを処理
for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
        file_path = os.path.join(folder_path, filename)
        
        # XMLデータを読み込む
        with open(file_path, "r", encoding="utf-8") as file:
            data = file.read()

        # XMLをパース
        root = ET.fromstring(data)

        # すべての要素をリスト化
        all_elements = list(root)

        # 各局を処理
        for i, elem in enumerate(all_elements):
            if elem.tag == "INIT":
                total_rounds += 1  # 局数をカウント
                
                furo_in_round = set()  # 集合型で副露者を記録することで、同局における同プレイヤーの2副露目以降のカウントに対処
                last_agari_tag = None  # ダブロン処理のための初期設定

                # 現在のINITタグから次のINITタグの手前までを調べる
                for sibling in all_elements[i + 1:]:
                    if sibling.tag == "INIT":
                        break  # 次の局に移ったら終了

                    # 副露者の情報を記録
                    if sibling.tag == "N":
                        # m属性（晒した牌の情報）を取得
                        m_value = sibling.get("m")

                        # m属性を16ビット整数値に変換
                        m_int = int(m_value)
                        m_binary = format(m_int, '016b')  # 16ビットの2進数表記に変換

                        # 1〜8ビット目がすべて0であればアンカンを示す
                        if m_binary[-8:] == "00000000":
                            continue  # アンカンは副露にカウントしない

                        # 副露タグ<N>に対応するwho属性を取得
                        who = sibling.get("who")
                        furo_in_round.add(who)  # 重複を避けるためにset型に追加

                    # リーチをカウント
                    if sibling.tag == "REACH":
                        # step="1"（リーチ宣言）をカウント（リーチ成立の有無は問わない）
                        if sibling.get("step") == "1":
                            total_reach += 1

                    # 放銃をカウント
                    if sibling.tag == "AGARI":
                        who = sibling.get("who")
                        from_who = sibling.get("fromWho")
                        # ダブロン処理：直前のAGARIタグと同じfromWhoが出てきた場合は無視する
                        if last_agari_tag is not None and last_agari_tag.get("fromWho") == from_who:
                            continue  # 2つ目のAGARIタグを無視
                        # whoとfromWhoが異なる場合に放銃としてカウント
                        if who != from_who:
                            total_houju += 1
                        # 現在のAGARIタグを保存
                        last_agari_tag = sibling

                # 1局中のユニークな副露をカウント
                total_furo += len(furo_in_round)
                
        # アガリ数をAGARIタグの数でカウント
        agari_tags = root.findall(".//AGARI")
        total_agari += len(agari_tags)

        # 流局数をRYUUKYOKUタグの数でカウント
        ryuukyoku_tags = root.findall(".//RYUUKYOKU")
        total_ryuukyoku += len(ryuukyoku_tags)

        # AGARIタグから打点を抽出してリストに格納
        for agari in root.findall(".//AGARI"):
            daten = agari.get("ten")
            daten_values = daten.split(",")
            datens.append(int(daten_values[1]))  # 2番目の値をリストに追加

        # UNタグから段位とrateを抽出してリストに格納
        for un in root.findall(".//UN"):
            dan = un.get("dan")
            rate = un.get("rate")
            if dan:  # UNタグ自体は牌譜ログに何度か発生することがあるので、dan属性が存在する場合のみ処理
                dan_values = dan.split(",")  # カンマで区切られたレートをリストに分割
                player_dans.extend([float(r) for r in dan_values])
            if rate:
                rate_values = rate.split(",")  # カンマで区切られたレートをリストに分割
                player_rates.extend([float(r) for r in rate_values])

# player_dansから9を引いて通常の段位表記に変換
player_dans = [int(x - 9) for x in player_dans]

# 副露やリーチのチャンス数（局数 × 4人分）
total_chances = total_rounds * 4
#　副露率・リーチ率・アガリ率・放銃率を計算
furo_rate = total_furo / total_chances
reach_rate = total_reach / total_chances
agari_rate = total_agari / total_chances
houju_rate = total_houju / total_chances
ryuukyoku_rate = total_ryuukyoku / total_rounds
# 平均アガリ打点を計算
average_daten = sum(datens) / len(datens)
# 平均段位を計算
average_dan = sum(player_dans) / len(player_dans)
# 7~11段の割合を計算
ratio_7dan = player_dans.count(7) / len(player_dans)
ratio_8dan = player_dans.count(8) / len(player_dans)
ratio_9dan = player_dans.count(9) / len(player_dans)
ratio_10dan = player_dans.count(10) / len(player_dans)
ratio_11dan = player_dans.count(11) / len(player_dans)
# 平均レートを計算
average_rate = sum(player_rates) / len(player_rates)
# レートの標準偏差を計算
sd_rate = statistics.stdev(player_rates)

In [ ]:
import csv

# 年を指定
year = 2023

# 結果データをリストにまとめる
results = [
    ["年", "対局数", "総局数", "副露回数", "副露率", "リーチ回数", "リーチ率", "アガリ数", "アガリ率",
     "放銃回数", "放銃率", "流局回数", "流局率", "和了打点",
     "平均段位", "7段の割合", "8段の割合", "9段の割合", "10段の割合", "11段の割合", "平均レート", "レートの標準偏差"],
    [year, total_games, total_rounds, total_furo, furo_rate, total_reach, reach_rate, total_agari, agari_rate,
     total_houju, houju_rate, total_ryuukyoku, ryuukyoku_rate, average_daten,
     average_dan, ratio_7dan, ratio_8dan, ratio_9dan, ratio_10dan, ratio_11dan, average_rate, sd_rate]
]

# CSVファイルに書き出す
csv_file = "output/2023.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(results)